<a href="https://colab.research.google.com/github/TheMihirNaik/google-search-console-api-using-python/blob/main/Plotting_Branded_vs_Non_branded_Queries_using_GSC_API_%26_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "Enter your Own Client ID"
CLIENT_SECRET = "Enter your Own Client Secret"
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print("Go to the following link in your browser: " + authorize_url)
auth_code = input("Enter your Authorization Code here:")
credentials = flow.step2_exchange(auth_code)
http = httplib2.Http()
creds = credentials.authorize(http)
webmasters_service = build('searchconsole', 'v1', http=creds)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=1085687348720-cpabg6ttpsqjji0j6pgbkb8sbcdcp4nt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter your Authorization Code here:4/1AVHEtk4s-dvol0vxJj7blvnYAST9zoV-OCCBkICTcTmkbaI0wCDt5vDNjmE


# List of Properties in GSC Account

In [ ]:
# Get a list of site in my Google Search Console Account
site_list = webmasters_service.sites().list().execute()

site_list

{'siteEntry': [{'siteUrl': 'https://www.tridentpbi.in/',
   'permissionLevel': 'siteFullUser'},
  {'siteUrl': 'https://www.monocubed.com/', 'permissionLevel': 'siteFullUser'},
  {'siteUrl': 'https://moneymonkdigital.com/',
   'permissionLevel': 'siteOwner'}]}

# Getting GSC Data & Preparing Dataframe using Pandas

In [ ]:
website = 'https://www.tridentpbi.in/'


# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2022-01-01',
      "endDate" : '2023-03-31',
      "dimensions" : ['DATE', 'QUERY'],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 193770 rows of data


In [ ]:
len(all_responses)

193770

In [ ]:
all_responses[0]

{'keys': ['2022-01-20', 'trident paper box industries'],
 'clicks': 8,
 'impressions': 8,
 'ctr': 1,
 'position': 1}

In [ ]:
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #date
  temp.append(each['keys'][0])

  #query
  temp.append(each['keys'][1])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)



In [ ]:
data_for_df[0]

['2022-01-20', 'trident paper box industries', 8, 8, 1, 1]

In [ ]:
import pandas as pd

df = pd.DataFrame(data_for_df, columns=['date', 'query', 'clicks', 'impressions', 'ctr', 'position'])

In [ ]:
df

,date,query,clicks,impressions,ctr,position
0,2022-01-20,trident paper box industries,8,8,1.000000,1.000000
1,2022-02-25,trident paper box industries,8,11,0.727273,1.000000
2,2022-07-25,trident paper box industries,8,11,0.727273,1.000000
3,2023-02-15,trident paper box industries,8,14,0.571429,1.142857
4,2023-03-01,trident paper box industries,8,9,0.888889,1.222222
...,...,...,...,...,...,...
193765,2023-03-31,white box manufacturers,0,3,0.000000,80.333333
193766,2023-03-31,white cardboard boxes,0,2,0.000000,69.000000
193767,2023-03-31,white carton,0,2,0.000000,1.000000
193768,2023-03-31,white corrugated boxes,0,3,0.000000,13.333333


# Preparing DF for Branded vs Non-branded Queries

In [ ]:
# function that determines whether the query is branded or non-branded
def keyword_type(query):
  # list of terms that defines branded queries
  brand_terms = ['trident', 'white']

  # identifying if the query is a branded query
  for term in brand_terms:
    if term in query:
      return 'brand'

  #if it's not a branded query, then it's a non_braned query
  return 'non_brand'

In [ ]:
# apply this function to the dataframe & create a new column - query_type (branded or non-branded)
df['keyword_type'] = df['query'].apply(keyword_type)

In [ ]:
df

,date,query,clicks,impressions,ctr,position,keyword_type
0,2022-01-20,trident paper box industries,8,8,1.000000,1.000000,brand
1,2022-02-25,trident paper box industries,8,11,0.727273,1.000000,brand
2,2022-07-25,trident paper box industries,8,11,0.727273,1.000000,brand
3,2023-02-15,trident paper box industries,8,14,0.571429,1.142857,brand
4,2023-03-01,trident paper box industries,8,9,0.888889,1.222222,brand
...,...,...,...,...,...,...,...
193765,2023-03-31,white box manufacturers,0,3,0.000000,80.333333,brand
193766,2023-03-31,white cardboard boxes,0,2,0.000000,69.000000,brand
193767,2023-03-31,white carton,0,2,0.000000,1.000000,brand
193768,2023-03-31,white corrugated boxes,0,3,0.000000,13.333333,brand


In [ ]:
# prepare a dataframe to plot a chart - groupby

plot_df = df.groupby(['date', 'keyword_type']).agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum'),
    ctr = ('ctr', 'mean'),
    position = ('position', 'mean'),
    count_queries = ('clicks', 'count'),
).reset_index()

In [ ]:
plot_df

,date,keyword_type,clicks,impressions,ctr,position,count_queries
0,2022-01-01,brand,1,11,0.142857,17.571429,7
1,2022-01-01,non_brand,10,428,0.020004,41.375847,242
2,2022-01-02,brand,3,15,0.055556,34.000000,6
3,2022-01-02,non_brand,11,498,0.016878,41.134455,276
4,2022-01-03,brand,1,13,0.050000,31.300000,10
...,...,...,...,...,...,...,...
905,2023-03-29,non_brand,5,911,0.007438,34.778039,484
906,2023-03-30,brand,5,36,0.064935,18.698701,11
907,2023-03-30,non_brand,4,835,0.003398,32.726785,427
908,2023-03-31,brand,0,19,0.000000,32.033333,10


In [ ]:
import plotly.express as px

fig = px.line(plot_df, x="date", y="clicks", color='keyword_type')

fig.show()

In [ ]:
import plotly.express as px

fig = px.line(plot_df, x="date", y="impressions", color='keyword_type')

fig.show()

In [ ]:
import plotly.express as px

fig = px.line(plot_df, x="date", y="ctr", color='keyword_type')

fig.show()

In [ ]:
import plotly.express as px

fig = px.line(plot_df, x="date", y="position", color='keyword_type')

fig.show()

In [ ]:
import plotly.express as px

fig = px.line(plot_df, x="date", y="count_queries", color='keyword_type')

fig.show()